# 1.6 een mandataris heeft steeds een startdatum

We load Javascript libraries to i) retrieve data (bindings) from a bestuurseenheid, ii) validate the document, and iii) display the JSON result.

In [1]:
import { getBestuurseenheidUrl } from "./utils.ts";
import {  getBindingsFromTurtleContent, validateDocument, fetchDocument } from 'npm:@lblod/lib-decision-validation';
import { display } from "https://deno.land/x/display/mod.ts";

const bestuurseenheidUuid = 'd168033a9bac278fa744c425e078eeabd304397f953feaaf5327b4e039aecacb';
const bestuurseenheidUrl = getBestuurseenheidUrl(bestuurseenheidUuid);
const bestuurseenheidBindings = await fetchDocument(bestuurseenheidUrl);

Below, we can find the SPARQL template query and SHACL shape for the 1.3 use case.
The Turtle string is converted into bindings.

In [3]:
const mandataris_1_6_query: string = `
    PREFIX besluit: <http://data.vlaanderen.be/ns/besluit#>
    PREFIX mandaat: <http://data.vlaanderen.be/ns/mandaat#>
    PREFIX org: <http://www.w3.org/ns/org#>
    PREFIX lmb: <http://lblod.data.gift/vocabularies/lmb/>

    select DISTINCT ($this as ?this) ?value
    where {
        $this a mandaat:Mandataris .
        FILTER NOT EXISTS {
            $this mandaat:start ?start .
        }
       
        BIND ("Mandataris heeft geen startdatum." as ?value)
    }`;
    
const mandataris_1_6_shape: string = `
@prefix sh: <http://www.w3.org/ns/shacl#> .

<http://example.org/mandataris_1_6_blueprint>
  a sh:NodeShape ;
  sh:targetClass <http://data.vlaanderen.be/ns/mandaat#Mandataris> ;
  sh:sparql [
		sh:select """${mandataris_1_6_query}""" ;
        sh:message 'Mandataris heeft geen startdatum.'
] .`;

const mandataris_1_6_bindings: Bindings[] = await getBindingsFromTurtleContent(mandataris_1_6_shape);

To test the shape, we retrieved a subject from the triple store that is applicable. In this case, we choose a mandataris URI.
Then, the template query from above is applied on this subject.
Next, a Comunica URL is generated to debug the results of this validation.

In [4]:
// Test subject
const subject = 'http://data.lblod.info/id/mandatarissen/4a979b34-e796-4db4-b102-f2e5b9b65b4d';
const mandataris_1_6_subject_query = mandataris_1_6_query.replaceAll('$this', `<${subject}>`);

const mandataris_1_6_comunica = `https://query.linkeddatafragments.org/#datasources=${encodeURIComponent(bestuurseenheidUrl)}&query=${encodeURIComponent(mandataris_1_6_subject_query)}`;
console.log(mandataris_1_6_comunica);

https://query.linkeddatafragments.org/#datasources=http%3A%2F%2Flocalhost%3A8890%2Fsparql%2F%3Fdefault-graph-uri%3D%26query%3D%250A%2520%2520%2520%2520construct%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520%253Fs%2520%253Fp%2520%253Fo%2520.%250A%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520where%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520graph%2520%253Chttp%253A%252F%252Fmu.semte.ch%252Fgraphs%252Forganizations%252Fd168033a9bac278fa744c425e078eeabd304397f953feaaf5327b4e039aecacb%252FLoketLB-mandaatGebruiker%253E%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%253Fs%2520%253Fp%2520%253Fo%2520.%250A%2520%2520%2520%2520%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520&query=%0A%20%20%20%20PREFIX%20besluit%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fbesluit%23%3E%0A%20%20%20%20PREFIX%20mandaat%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fmandaat%23%3E%0A%20%20%20%20PREFIX%20org%3A%20%3Chttp%3A%2F%2Fwww.w

The full validation on all mandatarissen happens below:

In [5]:
const mandataris_1_6_validationReport = await validateDocument(bestuurseenheidBindings, mandataris_1_6_bindings);
await display({ "application/json": mandataris_1_6_validationReport},{ raw: true });